# Payoff Forward-start

Strike = m x S_start.


In [ ]:

import matplotlib.pyplot as plt
import numpy as np
import ipywidgets as widgets
import sys
from pathlib import Path
from IPython.display import display, Markdown

_base = Path.cwd().resolve()
for extra in (_base, _base.parent, _base.parent.parent, _base.parent.parent.parent):
    candidate = extra / "scripts"
    if candidate.exists() and str(candidate) not in sys.path:
        sys.path.insert(0, str(candidate))
        break
    candidate = extra / "notebooks" / "scripts"
    if candidate.exists() and str(candidate) not in sys.path:
        sys.path.insert(0, str(candidate))
        break
plt.style.use('seaborn-v0_8-darkgrid')

from pricing import fetch_spy_history, view_forward_start, payoff_forward_start
close_spy = fetch_spy_history()
spot_ref = float(close_spy.iloc[-1])
spot_start = spot_ref
m_mult = 1.0


## Payoff interactif
Spot de depart, multiplicateur m, sigma, r, T, call/put.


In [ ]:

sigma_slider = widgets.FloatSlider(value=0.2, min=0.01, max=1.0, step=0.01, description='Sigma')
r_slider = widgets.FloatSlider(value=0.02, min=-0.05, max=0.1, step=0.005, description='r')
T_slider = widgets.FloatSlider(value=1.0, min=0.05, max=2.0, step=0.05, description='T')

spot_start_slider = widgets.FloatSlider(value=spot_start, min=spot_ref*0.6, max=spot_ref*1.4, step=1.0, description='Spot start')
spot_slider = widgets.FloatSlider(value=spot_ref, min=spot_ref*0.6, max=spot_ref*1.4, step=1.0, description='S0')
m_slider = widgets.FloatSlider(value=m_mult, min=0.5, max=1.5, step=0.05, description='Multiplicateur')
call_put = widgets.Dropdown(options=[('Call','call'),('Put','put')], value='call', description='Type')
output = widgets.Output()

def _update(change=None):
    with output:
        output.clear_output()
        s0, s_start, m = spot_slider.value, spot_start_slider.value, m_slider.value
        view_dyn = view_forward_start(s0, s_start, m=m, option_type=call_put.value, r=r_slider.value, q=0.0, sigma=sigma_slider.value, T=T_slider.value)
        premium = float(view_dyn.get('premium', 0.0))
        payoff_val = payoff_forward_start(s0, s_start, m=m, option_type=call_put.value)
        pnl_val = payoff_val - premium
        s_grid = view_dyn['s_grid']
        payoff_grid = view_dyn['payoff']
        pnl_grid = view_dyn['pnl']
        fig, ax = plt.subplots(figsize=(7,4))
        ax.plot(s_grid, payoff_grid, label='Payoff')
        ax.plot(s_grid, pnl_grid, label='P&L net', color='darkorange')
        ax.axvline(m * s_start, color='gray', linestyle='--', label=f'K = {m*s_start:.2f}')
        ax.axvline(s0, color='crimson', linestyle='-.', label=f'S0 = {s0:.2f}')
        ax.axhline(0, color='black', linewidth=0.8)
        ax.legend(loc='best')
        ax.set_xlabel('Spot')
        ax.set_ylabel('Payoff / P&L')
        ax.set_title('Forward-start')
        plt.show()
        plt.close(fig)
        display(Markdown(f"Prime ~ {premium:.4f}

- Payoff @ S0 = {payoff_val:.4f}
- P&L net = {pnl_val:.4f}"))

for sl in (spot_start_slider, spot_slider, m_slider, sigma_slider, r_slider, T_slider, call_put):
    sl.observe(_update, names='value')
_update()
display(widgets.VBox([spot_start_slider, spot_slider, m_slider, sigma_slider, r_slider, T_slider, call_put, output]))
